# McDonalds Diet Problem

In this notebook, we will determine the minimum cost McDonalds diet

In [1]:
foods = [:QP, :MD, :BM, :FF, :MC, :FR, :SM, :M1, :OJ]
# Word to the wise---Julia symbols cannot start with a number (M1 instead of 1M. It took me a while to figure this out)

nutrients = [:Prot, :VitA, :VitC, :Calc, :Iron, :Cals, :Carb]

cost = Dict(zip(foods,[1.84,2.19,1.84,1.44,2.29,0.77,1.29,0.6,0.72]))
required = Dict(zip(nutrients,[55,100,100,100,100,2000,350]))
using NamedArrays
A = [
    28 24 25 14 31 3 15 9 1
    15 15 6 2 8 0 4 10 2
    6 10 2 0 15 15 0 4 120
    30 20 25 15 15 0 20 30 2
    20 20 20 10 8 2 15 0 2
    510 370 500 370 400 220 345 110 80
    34 33 42 38 42 26 27 12 20
]
A_NA = NamedArray(A, (nutrients,foods), ("Nutrients","Menu Items")) ;

In [2]:
burgers = [:QP, :MD, :BM] ;

Suppose instead of minimizing cost, I wish to maximize the number of hamburgers I eat.  (I like hamburgers A LOT!)

In [3]:
using JuMP, HiGHS, Printf
m = Model(HiGHS.Optimizer)
@variable(m, x[foods] >= 0)
@objective(m, Max, sum(x[j] for j in burgers))
@constraint(m, [i in nutrients], sum(A_NA[i,j]*x[j] for j in foods) >= required[i])

# If you don't want the solver output, you can make it slient like this
#set_silent(m)
optimize!(m)

sol = Dict(j => value(x[j]) for j in foods if value(x[j]) > 1.0e-6)

@printf("\nMinimum cost menu is \$%.2f\n", objective_value(m))
for (key, value) in sol
    @printf("Eat %.2f of menu item %s\n", value, key)
end


Running HiGHS 1.9.0 (git hash: 66f735e60): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 5e+02]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [6e+01, 2e+03]
Presolving model
Problem status detected on presolve: Primal infeasible or unbounded
Solving the original LP with primal simplex to determine infeasible or unbounded
Using EKK primal simplex solver
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.1207498688e-08 Ph1: 7(1920); Du: 9(100.207) 0s
          3     5.0000000000e+01 Pr: 0(0); Du: 1(0.5) 0s
Using EKK primal simplex solver
  Iteration        Objective     Infeasibilities num(sum)
          3     5.0000006313e+01 Pr: 0(0); Du: 1(0.5) 0s
          3     5.0000000000e+01 Pr: 0(0); Du: 1(0.5) 0s
Model status        : Unbounded
Simplex   iterations: 3
Objective value     :  5.0000000000e+01
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.01
Solving linear system to compute prima

See that the Model status is Unbounded!
In this case the "solution" returned is garbage.
We need to make sure the Model Status is 'Optimal' in order to 'trust' the solution.
See https://jump.dev/JuMP.jl/stable/manual/solutions/ for manual page

In [4]:
opt_status = termination_status(m)
if opt_status != MOI.OPTIMAL
    println("Solver did not find optimal solution, status: ", opt_status)
end


Solver did not find optimal solution, status: DUAL_INFEASIBLE


Here DUAL_INFEASIBLE status means that the LP instance was UNBOUNDED.
(We will learn more about duality later in the class).

In general you should always check the termination_status.


In [5]:
max_item = Dict(j => 10 for j in foods)



Dict{Symbol, Int64} with 9 entries:
  :QP => 10
  :MD => 10
  :FR => 10
  :SM => 10
  :OJ => 10
  :BM => 10
  :M1 => 10
  :MC => 10
  :FF => 10

In [6]:
m = Model(HiGHS.Optimizer)
@variable(m, 0 <= x[j in foods] <= max_item[j])
@objective(m, Max, sum(x[j] for j in burgers))
@constraint(m, [i in nutrients], sum(A_NA[i,j]*x[j] for j in foods) >= required[i])

# If you don't want the solver output, you can make it slient like this
#set_silent(m)
optimize!(m)

sol = Dict(j => value(x[j]) for j in foods if value(x[j]) > 1.0e-6)

@printf("\nMinimum cost menu is \$%.2f\n", objective_value(m))
for (key, value) in sol
    @printf("Eat %.2f of menu item %s\n", value, key)
end

Running HiGHS 1.9.0 (git hash: 66f735e60): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 5e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+01, 1e+01]
  RHS    [6e+01, 2e+03]
Presolving model
0 rows, 0 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-7); columns 0(-9); elements 0(-58) - Reduced to empty
Solving the original LP from the solution after postsolve
Model status        : Optimal
Objective value     :  3.0000000000e+01
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00

Minimum cost menu is $30.00
Eat 10.00 of menu item QP
Eat 10.00 of menu item MD
Eat 10.00 of menu item FR
Eat 10.00 of menu item SM
Eat 10.00 of menu item BM
Eat 10.00 of menu item M1
Eat 10.00 of menu item MC
Eat 10.00 of menu item FF


## Next idea In the interest of extended my life, my wife has requested that I obey the following constraints:

1. Don't eat more than 3 sandwiches per day
2. Don't drink too much  (Always very good advice).  At most 3 drinks per day.
3. Don't eat more than 2 orders of french fries per day.


In [7]:
sammiches = [:QP, :MD, :BM, :FF, :MC, :SM]

# Let's just make a new model for this
m2 = Model(HiGHS.Optimizer)
@variable(m2, x[foods] >= 0)
@objective(m2, Min, sum(cost[j]*x[j] for j in foods))
@constraint(m2, [i in nutrients], sum(A_NA[i,j]*x[j] for j in foods) >= required[i])
# We can 'name' the constraints if we wish
@constraint(m2, MaxSammich, sum(x[j] for j in sammiches) <= 3)
@constraint(m2, MaxDrinks, x[:M1] + x[:OJ] <= 3)
@constraint(m2, MaxFF, x[:FF] <= 2)
optimize!(m2)

opt_status = termination_status(m2)
if opt_status != MOI.OPTIMAL
    println("Solver did not find optimal solution, status: ", opt_status)
end

Running HiGHS 1.9.0 (git hash: 66f735e60): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 5e+02]
  Cost   [6e-01, 2e+00]
  Bound  [0e+00, 0e+00]
  RHS    [2e+00, 2e+03]
Presolving model
9 rows, 9 cols, 66 nonzeros  0s
9 rows, 9 cols, 66 nonzeros  0s
Presolve : Reductions: rows 9(-1); columns 9(-0); elements 66(-1)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 7(1420) 0s
          5     2.4751267377e+01 0s
Model status        : Infeasible
Simplex   iterations: 5
Objective value     :  2.4751250000e+01
HiGHS run time      :          0.00
Solving LP to try to compute dual ray
Coefficient ranges:
  Matrix [1e+00, 5e+02]
  Cost   [0e+00, 0e+00]
  Bound  [0e+00, 0e+00]
  RHS    [2e+00, 2e+03]
Solving LP without presolve, or with basis, or unconstrained
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num

* Suppose I think that the ``too much drinking'' constraint is the one causing the problem to be infeasible

* Let's create a new problem to determine if I had more than 3 drinks, the instance could be feasible 

* $s$:  Number of extra drinks (over three)
that I must drink in order to get a feasible solution
$$x_{1M} + x_{OJ} - s \leq 3, s \geq 0$$
* New Objective: $\min s$

In [8]:
using Printf
# We can delete constraints from a model 
delete(m2,MaxDrinks)
# Add a new variable
@variable(m2, s >= 0)
@constraint(m2, NewMaxDrinks, x[:M1] + x[:OJ] - s <= 3)
# This will over-write the old objective in model m2
@objective(m2, Min, s)

optimize!(m2)
opt_status = termination_status(m2)
if opt_status != MOI.OPTIMAL
    println("Solver did not find optimal solution, status: ", opt_status)
else
    @printf("I have to drink %.2f (extra) drinks\n", objective_value(m2))
    @printf("%.2f milk, %.2f OJ", value(x[:M1]), value(x[:OJ]))
end

Coefficient ranges:
  Matrix [1e+00, 5e+02]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [2e+00, 2e+03]
Presolving model
4 rows, 9 cols, 25 nonzeros  0s
4 rows, 9 cols, 25 nonzeros  0s
Presolve : Reductions: rows 4(-6); columns 9(-1); elements 25(-43)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2(125) 0s
          3     2.5000000000e+00 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 3
Objective value     :  2.5000000000e+00
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00
I have to drink 2.50 (extra) drinks
5.50 milk, 0.00 OJ